In [3]:
import pandas as pd

In [5]:
df=pd.read_csv('./Melbourne_housing_FULL.csv')
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


<data description 데이터 변수 정보>

• Rooms: Number of rooms (방의 갯수)

​

• Type(주택 형태):

br — bedroom(s);

h — house,cottage,villa, semi,terrace;

u — unit, duplex;

t — townhouse;

dev site — development site;

o res — other residential.

​

• Price: Price in Australian dollars (가격, 호주 달러)

​

• Method(소유권 상태, 경매 현황):

S — property sold;

SP — property sold prior;

PI — property passed in;

PN — sold prior not disclosed;

SN — sold not disclosed;

NB — no bid;

VB — vendor bid;

W — withdrawn prior to auction;

SA — sold after auction;

SS — sold after auction price not disclosed.

N/A — price or highest bid not available.

​

• Postcode: 우편번호

​

• Distance(CBD 로부터의 거리, km): Distance from CBD in Kilometres

​

• Regionname(지역명): General Region (West, North West, North, North east …etc)

​

• Propertycount(근접한 부동산수): Number of properties that exist in the suburb.

​

• CouncilArea(관청이 있는 지역): Governing council for the area

[출처] [데이터사이언스] 머신러닝 변수 선택 feature selecting — SelectKBest 사례 연습(주택가격 데이터를 바탕으로)|작성자 Jalynne



# 2 . 변수선택(feature selecting)하기

​

위의 변수들만 살펴봤을 때에는, 예측하고자 하는 타겟변수인 ‘가격’에 가장 주요하게 영향을 미치는 변수가 무엇일지 선택하기에 어려워보입니다. 

또한, 변수들 중 문자로 되어 있는 카테고리형(Categorical) 데이터를 수치형으로 치환하는 것이 필요해보입니다. 예를 들어, type(주택의 형태)의 경우 형태별로 가격대의 차이가 있을 것이므로 분석이 필요한데 데이터가 문자로 되어 있어 수치형으로 변환하여 분석하는 것이 필요함을 확인하실 수 있습니다.

## 1단계 : 카테고리형 데이터를 one-hot-encoding하기

In [8]:
from category_encoders import OneHotEncoder
encoder = OneHotEncoder(use_cat_names = True)
df1 = encoder.fit_transform(df)

MemoryError: Unable to allocate 9.06 GiB for an array with shape (34882, 34857) and data type int64

> 인코딩 결과: 카테고리형 데이터(ex. Type) 의 값들(ex. Type_h, Typed_t, Type_u)이 칼럼(column)으로 들어가고, 로우(row)에는 카테고리 데이터가 수치로(1 / 0 ) 변환된 것을 확인할 수 있습니다.


## ​2단계: 훈련데이터(train set) / 실험데이터(test set) 나누기

예측모델을 학습시킬 훈련데이터와 실험데이터를 나눠보겠습니다.

In [9]:
from sklearn.model_selection import train_test_split
X = df1[df1.columns.difference(['Price'])]
y = df1[['Price']]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

NameError: name 'df1' is not defined

> 결과값: 아래 결과값을 보니 훈련데이터(34,012 개)와 실험데이터(8,503개)가 잘 나누어진 것 같습니다. (전체 42,515개)

## 3단계: SelectKBest 모듈 활용하여 변수 선택(feature selecting) 하기.

scikit-learn의 SelectKBest 모듈은 target 변수와 그외 변수 사이의 상관관계를 계산하여 가장 상관관계가 높은 변수 k개를 선정할 수 있는 모듈입니다. 상관관계를 분석하는 방법은 f-regression 방식과 chi2(카이제곱)방식, f-classif 등이 있습니다. 본 분석에서는 f-regression을 사용했습니다.

In [ ]:
# target(Price)와 가장 correlated 된 features 를 k개 고르기.
## f_regresison, SelectKBest 불러오기.
from sklearn.feature_selection import f_regression, SelectKBest
## selctor 정의하기.
selector = SelectKBest(score_func=f_regression, k=20)
## 학습데이터에 fit_transform 
X_train_selected = selector.fit_transform(X_train, y_train)
## 테스트 데이터는 transform
X_test_selected = selector.transform(X_test)
X_train_selected.shape, X_test_selected.shape

위 결과값을 보시면, one-hot-encoding으로 인해 X_train / X_test 데이터의 컬럼수가 54개까지 늘어났던 컬럼수가 20개로 줄어든 것을 확인할 수 있습니다. 

SelectKbest모듈이 타겟변수와 상관관계가 높은 20개 컬럼을 선정한 것입니다.

In [ ]:
all_names = X_train.columns
## selector.get_support()
selected_mask = selector.get_support()
## 선택된 특성(변수)들
selected_names = all_names[selected_mask]
## 선택되지 않은 특성(변수)들
unselected_names = all_names[~selected_mask]
print('Selected names: ', selected_names)
print('Unselected names: ', unselected_names)